# **FEATURE SELECTION**

In [15]:
import numpy as np
import random
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Dataset
X, y = load_breast_cancer(return_X_y=True)
Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.2, random_state=42)

n_features = X.shape[1]
print("Número de características:", n_features)

Número de características: 30


In [16]:
# Cromosoma: vector binario de longitud n_features
#Generar población inicial
def random_chrom():
    return [random.choice([0,1]) for _ in range(n_features)]

#Evalúa la calidad de un cromosoma
def fitness(chrom):
    mask = np.array(chrom, dtype=bool)
    if mask.sum() == 0:  # evita seleccionar 0 columnas
        return 0
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(Xtr[:,mask], ytr)
    return clf.score(Xval[:,mask], yval)
#Seleccionar los mejores
def tournament_selection(pop, fitnesses, k=3):
    selected = []
    for _ in range(len(pop)):
        aspirants = random.sample(list(range(len(pop))), k)
        winner = max(aspirants, key=lambda i: fitnesses[i])
        selected.append(pop[winner])
    return selected
# Cruzar pares de padres
def one_point_crossover(p1, p2):
    point = random.randint(1, n_features-1)
    return p1[:point]+p2[point:], p2[:point]+p1[point:]
#Mutar ligeramente
def mutate(chrom, pm=0.1):
    for i in range(len(chrom)):
        if random.random() < pm:
            chrom[i] = 1 - chrom[i]
    return chrom

In [18]:
POP_SIZE = 6 #Cromosomas
GENERATIONS = 5

# Inicialización
pop = [random_chrom() for _ in range(POP_SIZE)]
print("Población inicial:", pop)

fitnesses = [fitness(ch) for ch in pop]

# Evolución
for gen in range(GENERATIONS):
    print(f"\n=== Generación {gen} ===")
    for i, (chrom, fit) in enumerate(zip(pop, fitnesses)):
        print(f" Individuo {i}: {chrom} → Fitness={fit:.4f}")

    parents = tournament_selection(pop, fitnesses, k=3)

    next_pop = []
    for i in range(0, POP_SIZE, 2):
        c1, c2 = one_point_crossover(parents[i], parents[(i+1)%POP_SIZE])
        next_pop += [c1, c2]

    next_pop = [mutate(c) for c in next_pop]
    print("   Después de mutación:", next_pop)

    fitnesses = [fitness(ch) for ch in next_pop]
    pop = next_pop

# Mejor solución
best_idx = np.argmax(fitnesses)
print("\n=== Mejor solución encontrada ===")
print("Cromosoma:", pop[best_idx])
print("Fitness:", fitnesses[best_idx])
print("Características seleccionadas:", np.where(np.array(pop[best_idx])==1)[0])

Población inicial: [[1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1], [0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1], [1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0], [1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1]]

=== Generación 0 ===
 Individuo 0: [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1] → Fitness=0.9474
 Individuo 1: [0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1] → Fitness=0.9474
 Individuo 2: [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1] → Fitness=0.9474
 Individuo 3: [1, 0, 1, 0, 0, 1, 1, 1, 0